In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2662, done.
remote: Counting objects: 100% (1175/1175), done.
remote: Compressing objects: 100% (359/359), done.
remote: Total 2662 (delta 918), reused 864 (delta 816), pack-reused 1487
Receiving objects: 100% (2662/2662), 2.03 MiB | 26.67 MiB/s, done.
Resolving deltas: 100% (1671/1671), done.


In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
    !mkdir ./Training
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.

# Training

In [ ]:
!wget https://huggingface.co/colbert-ir/colbertv2.0_msmarco_64way/resolve/main/examples.json #gets the original triples from colbertv2 to use in the destination language
!wget 'https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/collections/french_collection.tsv'
!wget 'https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/train/french_queries.train.tsv'

--2024-02-27 12:09:34--  https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/collections/portuguese_collection.tsv
Resolving huggingface.co (huggingface.co)... 13.35.7.5, 13.35.7.38, 13.35.7.57, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.5|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/unicamp-dl/mmarco/bd0074d22ec0deacb08e9f9851f8f3aeb6d03b0d59b1b05f926ad4961a229520?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27portuguese_collection.tsv%3B+filename%3D%22portuguese_collection.tsv%22%3B&response-content-type=text%2Ftab-separated-values&Expires=1709294974&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTI5NDk3NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy91bmljYW1wLWRsL21tYXJjby9iZDAwNzRkMjJlYzBkZWFjYjA4ZTlmOTg1MWY4ZjNhZWI2ZDAzYjBkNTliMWIwNWY5MjZhZDQ5NjFhMjI5NTIwP3Jlc3BvbnNlLWNvbnRlbnQtZGl

In [ ]:
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from datasets import load_dataset

In [ ]:
checkpoint = 'antoinelouis/colbertv1-camembert-base-mmarcoFR'
language = 'french'

In [ ]:
from colbert.infra.run import Run
from colbert.infra.config import ColBERTConfig, RunConfig
from colbert import Trainer


def train():
    # use 4 gpus (e.g. four A100s, but you can use fewer by changing nway,accumsteps,bsize).
    with Run().context(RunConfig(nranks=1, name='ColBERTv2.0-camembert-based-mmarcoFR', avoid_fork_if_possible=True, root = '/content/drive/MyDrive/Colab Notebooks/french-checkpoints')):
        triples = 'examples.json'
        queries = f'{language}_queries.train.tsv'
        collection = f'{language}_collection.tsv'

        config = ColBERTConfig(
            bsize=64,
            lr=1e-05,
            warmup=20_000,
            doc_maxlen=180,
            dim=128,
            attend_to_mask_tokens=False,
            nway=4,
            accumsteps=1,
            similarity='cosine',
            use_ib_negatives=True,
            maxsteps=200000,
            save_every=10000,
            )

        trainer = Trainer(triples=triples, queries=queries, collection=collection, config=config)

        trainer.train(checkpoint=checkpoint)  # or start from scratch, like `bert-base-uncased`


if __name__ == '__main__':
    train()

{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 1,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 128,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 200000,
    "save_every": 10000,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 180,
    "mask_punctuation": true,
    "checkpoint": "antoinelouis\/colbertv1-camembert-base-mmarcoFR",
    "triples": "\/content\/drive\/MyDrive\/Colab Notebooks\/data_colbert\/examples.j

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Streaming output truncated to the last 5000 lines.
#>>>    21.1 15.64 		|		 5.460000000000001
[Feb 27, 13:56:58] 86 0.3488790604763124
				 0.1826304793357849 0.12982921302318573
#>>>    21.47 16.53 		|		 4.939999999999998
[Feb 27, 13:56:58] 87 0.3488426411230962
				 0.15792851150035858 0.15008465945720673
#>>>    21.43 16.34 		|		 5.09
[Feb 27, 13:56:59] 88 0.34880181165293067
				 0.14866763353347778 0.1542704999446869
#>>>    21.56 15.68 		|		 5.879999999999999
[Feb 27, 13:56:59] 89 0.3487559479747559
				 0.21860460937023163 0.1162266954779625
#>>>    21.08 15.95 		|		 5.129999999999999
[Feb 27, 13:57:00] 90 0.34874202332417875
				 0.2821371555328369 0.20317929983139038
#>>>    20.82 16.44 		|		 4.379999999999999
[Feb 27, 13:57:00] 91 0.34887859775621877
				 0.1622951179742813 0.14649119973182678
#>>>    21.01 15.98 		|		 5.030000000000001
[Feb 27, 13:57:00] 92 0.3488385054910698
				 0.22530049085617065 0.13478927314281464
#>>>    21.21 15.93 		|		 5.280000000000001
[Feb 27, 13

KeyboardInterrupt: 

In [ ]:
from huggingface_hub import HfApi
from google.colab import userdata

hf_api = HfApi(token=userdata.get('HF_TOKEN2'))

In [ ]:
hf_api.create_repo(
    repo_id='AdrienB134/ColBERTv2.0-camembert-based-mmarcoFR',

)



NameError: name 'hf_api' is not defined

In [ ]:
hf_api.upload_folder(
    repo_id='AdrienB134/ColBERTv2.0-camembert-based-mmarcoFR',
    folder_path='/content/experiments/default/none/AdrienB134/ColBERTv2.0-camembert-based-mmarcoFR/checkpoints/colbert',
    )

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AdrienB134/ColBERTv2.0-spanish-mmarcoES/commit/345ca60a770eb3fac8ad3276d0ea1cf9cd5c8f5c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='345ca60a770eb3fac8ad3276d0ea1cf9cd5c8f5c', pr_url=None, pr_revision=None, pr_num=None)

# Evaluation

In [ ]:
!wget 'https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/qrels.dev.small.tsv'
!wget 'https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/dev/french_queries.dev.small.tsv'

In [ ]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Indexer
collection = f'{language}_collection.tsv'
with Run().context(RunConfig(nranks=1, experiment="msmarco", avoid_fork_if_possible = True, overwrite=True)):

    config = ColBERTConfig(
          nbits=2,
          root="./experiments",
        )
    indexer = Indexer(checkpoint="AdrienB134/ColBERTv2.0-camembert-based-mmarcoFR", config=config)
    indexer.index(name="msmarco.nbits=2", collection=collection)

artifact.metadata:   0%|          | 0.00/1.71k [00:00<?, ?B/s]



[Feb 27, 08:06:09] #> Creating directory /content/experiments/msmarco/indexes/msmarco.nbits=2 


{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 200000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 180,
    "mask_punctuation": true,
    "checkpoint": "AdrienB134\/ColBERTv2.0-spanish-mmarc

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

...

100%|██████████| 354/354 [00:00<00:00, 392.03it/s]

[Feb 27, 10:36:30] [0] 		 Sorting codes...


[Feb 27, 10:37:46] [0] 		 Getting unique codes...
[Feb 27, 10:37:47] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 27, 10:37:47] #> Building the emb2pid mapping..
[Feb 27, 10:38:15] len(emb2pid) = 667011457


100%|██████████| 262144/262144 [02:04<00:00, 2100.57it/s]


[Feb 27, 10:40:28] #> Saved optimized IVF to /content/experiments/msmarco/indexes/msmarco.nbits=2/ivf.pid.pt
[Feb 27, 10:40:28] [0] 		 #> Saving the indexing metadata to /content/experiments/msmarco/indexes/msmarco.nbits=2/metadata.json ..


In [ ]:
from colbert.data import Queries
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Searcher

if __name__=='__main__':
    with Run().context(RunConfig(nranks=1, experiment="msmarco", avoid_fork_if_possible=True)):

        config = ColBERTConfig(
            root="./experiments",
        )
        searcher = Searcher(index="msmarco.nbits=2", config=config)
        queries = Queries(f"{language}_queries.dev.small.tsv")
        ranking = searcher.search_all(queries, k=10000)
        ranking.save(f"colbertv2-{language}-final.ranking.tsv")

[Feb 27, 10:40:29] #> Loading collection...
0M 1M 2M 3M 4M 5M 6M 7M 8M 
[Feb 27, 10:41:15] #> Loading codec...
[Feb 27, 10:41:15] #> Loading IVF...
[Feb 27, 10:41:16] #> Loading doclens...


100%|██████████| 354/354 [00:00<00:00, 1115.01it/s]


[Feb 27, 10:41:17] #> Loading codes and residuals...


100%|██████████| 354/354 [00:19<00:00, 18.29it/s]


[Feb 27, 10:41:36] #> Loading the queries from /content/drive/MyDrive/Colab Notebooks/data_colbert/spanish_queries.dev.small.tsv ...
[Feb 27, 10:41:37] #> Got 6980 queries. All QIDs are unique.



6980it [01:50, 62.93it/s]




[Feb 27, 10:43:35] #> Creating directory /content/experiments/msmarco/none/2024-02/27/08.06.06 


[Feb 27, 10:43:37] #> Saved ranking of 6980 queries and 698000 lines to /content/drive/MyDrive/Colab Notebooks/data_colbert/colbertv2-spanish-final.ranking.tsv


In [ ]:
!python -m ColBERT.utility.evaluate.msmarco_passages --ranking "colbertv2-french-final.ranking.tsv" --qrels "qrels.dev.small.tsv"